In [15]:
!pip install pyproj

  Using cached certifi-2025.7.14-py3-none-any.whl.metadata (2.4 kB)
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 6.3/6.3 MB 77.1 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [pyproj]
   -------------------- ------------------- 1/2 [pyproj]
   -------------------- ------------------- 1/2 [pyproj]
   ---------------------------------------- 2/2 [pyproj]



In [25]:
import pandas as pd
from pyproj import Transformer

In [26]:
df = pd.read_csv("daejeon_local_hospital.csv", encoding='utf-8')
df.columns.to_list()

['번호',
 '개방서비스명',
 '개방서비스아이디',
 '개방자치단체코드',
 '관리번호',
 '인허가일자',
 '영업상태명',
 '상세영업상태코드',
 '상세영업상태명',
 '소재지전화',
 '소재지우편번호',
 '소재지전체주소',
 '도로명전체주소',
 '도로명우편번호',
 '사업장명',
 '최종수정시점',
 '데이터갱신구분',
 '데이터갱신일자',
 '업태구분명',
 '좌표정보x(epsg5174)',
 '좌표정보y(epsg5174)',
 '의료기관종별명',
 '의료인수',
 '입원실수',
 '병상수',
 '총면적',
 '진료과목내용',
 '진료과목내용명',
 '지정취소일자',
 '완화의료지정형태']

In [27]:
df[['사업장명', '좌표정보y(epsg5174)', '좌표정보x(epsg5174)']].head(15)

,사업장명,좌표정보y(epsg5174),좌표정보x(epsg5174)
0,대전대학교 대전한방병원,316257.3029,233980.0617
1,의료법인 원전의료재단 가양요양병원,316259.2286,240048.7929
2,의료법인 영훈의료재단 선치과병원,315310.5702,237098.1477
3,근로복지공단 대전병원,318721.2539,238383.2615
4,에이스마일치과병원,321248.8281,227923.3556
5,백세요양병원,313754.0904,238385.2675
6,새봄요양병원,312452.6884,241043.8249
7,강남한방병원,313785.6328,235289.3598
8,중구의료소비자생활협동조합 보문산생태요양병원,312345.5078,237714.0041
9,둔산엔젤요양병원,317142.6906,234034.7927


In [ ]:
x_point = df['좌표정보x(epsg5174)'].astype(float).to_list()  # 경도
y_point = df['좌표정보y(epsg5174)'].astype(float).to_list()  # 위도

# EPSG:5174는 국내 행정지도 측량용 좌표
# EPSG:4326은 전 세계 지도 서비스 표준 좌표
# 좌표 변환기 정의, epsg:5174 -> epsg:4326 # 변환 규칙 명시
transformer = Transformer.from_crs("epsg:5174", "epsg:4326", always_xy=True)

lon_list, lat_list = transformer.transform(x_point, y_point)

df['경도_X(epsg4326)'] = lon_list
df['위도_Y(epsg4326)'] = lat_list

# 기존 좌표계 삭제
df.drop(columns=['좌표정보x(epsg5174)', '좌표정보y(epsg5174)'], inplace=True)

# 변환된 데이터 확인
df[['사업장명', '위도_Y(epsg4326)', '경도_X(epsg4326)']].head(3) # 구글 대차비교용 출력

,사업장명,위도_Y(epsg4326),경도_X(epsg4326)
0,대전대학교 대전한방병원,36.346538,127.379329
1,의료법인 원전의료재단 가양요양병원,36.346322,127.446932
2,의료법인 영훈의료재단 선치과병원,36.337892,127.414018


# 대전대학교 대전한방병원	| 36.346538 | 127.379329
# https://www.google.com/maps?q=36.34653792451366,127.37932946833648
# 구글에 경우 위도(y), 경도(x) 순서로 입력.
# 네이버는 api 변수 입력 규칙 문서 참조 필요.

In [30]:
df.to_csv("(conversion)_daejeon_local_hospital_transformed.csv", index=False, encoding='utf-8')

In [32]:
saveing_check_df = pd.read_csv("(conversion)_daejeon_local_hospital_transformed.csv", encoding='utf-8')
saveing_check_df.columns.to_list()

['번호',
 '개방서비스명',
 '개방서비스아이디',
 '개방자치단체코드',
 '관리번호',
 '인허가일자',
 '영업상태명',
 '상세영업상태코드',
 '상세영업상태명',
 '소재지전화',
 '소재지우편번호',
 '소재지전체주소',
 '도로명전체주소',
 '도로명우편번호',
 '사업장명',
 '최종수정시점',
 '데이터갱신구분',
 '데이터갱신일자',
 '업태구분명',
 '의료기관종별명',
 '의료인수',
 '입원실수',
 '병상수',
 '총면적',
 '진료과목내용',
 '진료과목내용명',
 '지정취소일자',
 '완화의료지정형태',
 '경도_X(epsg4326)',
 '위도_Y(epsg4326)']

In [33]:
# 컨버전 저장 확인용 출력
saveing_check_df[['사업장명', '위도_Y(epsg4326)', '경도_X(epsg4326)']].head(3)  

,사업장명,위도_Y(epsg4326),경도_X(epsg4326)
0,대전대학교 대전한방병원,36.346538,127.379329
1,의료법인 원전의료재단 가양요양병원,36.346322,127.446932
2,의료법인 영훈의료재단 선치과병원,36.337892,127.414018
